Log in to LinkedIn?



In [ ]:
var notRobot = () => {
    return client.click('')
}

var enterLinkedIn = () => {
    console.log('LinkedIn: Sign in required');
    var credentials = getCredentials('linkedin.com');
    return client.click('input[name="session_key"]')
        .keys(credentials.session_key)
        .pause(1000)
        .then(() => console.log('LinkedIn: Require password'))
        .click('input[name="session_password"]')
        .keys(credentials.session_password)
        .submitForm('.login-form');
}

var loginLinkedIn = () => {
    return client.url('https://www.linkedin.com/')
        .pause(1000)
        .isExisting('*=Forgot password?').then(is => is 
            ? enterLinkedIn()
            : Promise.resolve([]));
};
module.exports = loginLinkedIn;
loginLinkedIn;


Scrape my own LinkedIn profile?



In [ ]:
var visitMyProfile = () => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client.click('#nav-settings__dropdown-trigger')
        .pause(500)
        .click('.nav-settings__view-profile-link')
        .pause(2000);
}

var readLinkedInProfileInfo = () => {
    var name, title, url, phone, email;
    return client
        .getUrl().then(url => url.indexOf('/in/') == -1
            ? visitMyProfile()
            : Promise.resolve([]))
        .click('.contact-see-more-less')
        .then(() => Promise.all([
            client.getText('.pv-top-card-section__name'),
            client.getText('.pv-top-card-section__headline'),
            client.getText('.ci-vanity-url .pv-contact-info__contact-item'),
            // is existing?
            client.getText('.ci-phone .pv-contact-info__contact-item'),
            client.getText('.ci-email .pv-contact-info__contact-item')
        ]))
        .then(r => ({
            name: r[0],
            title: r[1],
            url: r[2],
            phone: r[3],
            email: r[4],
        }));
};
module.exports = readLinkedInProfileInfo;
readLinkedInProfileInfo;


List all LinkedIn threads?



In [ ]:
var importer = require('../Core');

var scrollLinkedInMessages = () => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .click('.msg-conversations-container')
        .keys('PageDown')
        .keys('PageDown')
        .keys('PageDown')
        .keys('PageDown')
        .keys('PageDown')
        .pause(1000)
        .then(() => getLinkedInThreads());
}

var getLinkedInThreads = () => {
    var threads = [];
    return client    
        .execute(() => {
            var iterator = document.evaluate(
                '//*[@data-control-name="view_message"]/@href',
                document, null,
                XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
            var co = []; var m;
            while ((m = iterator.iterateNext()) && co.push(m.value));
            return co;
        })
        .then(els => els.value.map(el => el.value)
              .filter(e => threads.indexOf(e) === -1))
        .then(newThreads => {
            threads = threads.concat(newThreads);
            return newThreads.length > 0
                ? scrollLinkedInMessages()
                : Promise.resolve(threads);
        });
}

var listLinkedInThreads = () => {
    return client.getUrl().then(url => url.indexOf('/messaging') == -1
            ? client.click('a[href*="/messaging/"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getLinkedInThreads())
};
module.exports = listLinkedInThreads;
listLinkedInThreads;


In [ ]:
var importer = require('../Core');
$$.async()
var runSeleniumCell;
importer.interpretAll(['selenium cell'])
    .then(r => {
        runSeleniumCell = r[0].runInNewContext();
        return runSeleniumCell('log in linkedin');
    })
    .then(loginLinkedIn => loginLinkedIn())
    //.then(() => runSeleniumCell('scrape LinkedIn profile'))
    //.then(readLinkedInProfileInfo => readLinkedInProfileInfo())
    // show my profile info
    //.then(r => console.log(r))
    .then(() => runSeleniumCell('list all LinkedIn threads'))
    .then(listLinkedInThreads => listLinkedInThreads())
    // list all message threads
    .then(r => console.log(r))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
